In [120]:
import bs4
import lxml
import pandas as pd
import urllib
from urllib import request
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil
import re
import numpy as np

In [53]:
#Récupération des liens des pages
url = "http://www.caveaudelahuchette.fr/"
site = request.urlopen(url).read()
page = bs4.BeautifulSoup(site, 'lxml')
div = page.find_all('ul')
menu = div[1]                                                                    #On récupère le code html du menu principal
items = menu.find_all('li',class_= re.compile("concert", flags=re.IGNORECASE))   #On utilise regex pour récupérer tous les lignes qui contiennet le terme "concert"
liens ={}
for item in items:
    liens[str(item.get('class')[0].split('_')[1])] = url + item.a.get('href')
                                                                                  #On a récupéré les liens de la programmation pour chaques mois
for url in liens.values():
    web = request.urlopen(url).read()
    page = bs4.BeautifulSoup(web,'lxml')
    




dict_values(['http://www.caveaudelahuchette.fr//1/concerts_novembre_2023_1482427.html', 'http://www.caveaudelahuchette.fr//1/concerts_decembre_2023_1482445.html', 'http://www.caveaudelahuchette.fr//1/concerts_janvier_2024_1482553.html'])
http://www.caveaudelahuchette.fr//1/concerts_novembre_2023_1482427.html
http://www.caveaudelahuchette.fr//1/concerts_decembre_2023_1482445.html
http://www.caveaudelahuchette.fr//1/concerts_janvier_2024_1482553.html


In [132]:
#Fonction pour changer les dates
def get_days(string):
    figures = re.findall(r'\d+', string)
    if 'au' in string:
        days = np.arange(int(figures[0]),int(figures[1])+1)
    elif 'et' in string:
        days = np.array([int(figures[0]),int(figures[1])])
    else:
        days = np.array([int(figures[0])])
    return days
    

get_days('Mardi 28 novembre au 43')

array([28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43])

In [140]:
#Pour une page
url = "http://www.caveaudelahuchette.fr//1/concerts_novembre_2023_1482427.html"
site = request.urlopen(url).read()
page = bs4.BeautifulSoup(site, 'lxml')
balise_p = page.find_all('p')

horaires =(balise_p[1].text.split()[7][:5], balise_p[1].text.split()[24][:5])     #Récupération des heures de concert sous le format (heure en semaine, heure le week-end)

programmation = balise_p[2].get_text(separator = '%', strip = True).split('%')


information = ['nom', 'date','heure','prix','etablissement','adresse','arrondissement']
D ={key:[] for key in information}
for concert in programmation:
    carac = concert.split(' : ')
    days = get_days(carac[0])
    for day in days:
        D['date'].append( str(day) + ' novembre')
        D['nom'].append(carac[1])

D['prix'] = ['k']*len(D['nom'])
D['etablissement'] = ['Caveau de la Huchette']*len(D['nom'])
D['heure'] =['21h30']*len(D['nom'])
D['adresse'] = ['5 rue de la Huchette']*len(D['nom'])
D['arrondissement'] = ['5']*len(D['nom'])
df = pd.DataFrame(D)
print(df['date'])


0      1 novembre
1      2 novembre
2      3 novembre
3      4 novembre
4      5 novembre
5      6 novembre
6      7 novembre
7      8 novembre
8      9 novembre
9     10 novembre
10    11 novembre
11    12 novembre
12    13 novembre
13    14 novembre
14    15 novembre
15    16 novembre
16    17 novembre
17    18 novembre
18    19 novembre
19    20 novembre
20    21 novembre
21    22 novembre
22    23 novembre
23    24 novembre
24    25 novembre
25    26 novembre
26    27 novembre
27    28 novembre
28    29 novembre
29    30 novembre
Name: date, dtype: object


In [110]:
#L'international
url_2 = "https://www.linternational.fr/agenda"
request_text = request.urlopen(url_2).read()
page = bs4.BeautifulSoup(request_text, "lxml")
exhibitions = page.find_all('div', class_ = "Zz5cHg")
information = ['nom', 'date','heure','prix','etablissement','adresse','arrondissement']
D = {key:[] for key in information}
id = 0
for exhibition in exhibitions:
    info = exhibition.img.get('alt').split(' - ')
    D['nom'].append(id)
    D['date'].append(info[0])
    D['heure'].append(info[1])
    D['prix'].append(info[2])
    id = id + 1
n = len(D['date'])

p = []                                 
for prix in D['prix']:
    p.append(prix[:-1])
D['prix'] = p

D['etablissement'] = ["L'international"]*n
D['adresse'] = ["5 rue Moret"]*n
D['arrondissement'] = [11]*n

data = pd.DataFrame(D)
data.head()
#international_csv = data.to_csv('C:\\Users\\LENOVO\\Desktop\\data_international.csv', index =False, encoding = 'utf-8')



,nom,date,heure,prix,etablissement,adresse,arrondissement
0,0,Vendredi 24 novembre,20h,8,L'international,5 rue Moret,11
1,1,Vendredi 24 novembre,00h,7,L'international,5 rue Moret,11
2,2,Samedi 25 novembre,20h,8,L'international,5 rue Moret,11
3,3,Samedi 25 novembre,00h,7,L'international,5 rue Moret,11
4,4,Dimanche 26 novembre,18h,Gratui,L'international,5 rue Moret,11


In [16]:
#Tentative de recup des covers
url_3 = "https://www.linternational.fr/agenda"
request_text = request.urlopen(url_3).read()
page = bs4.BeautifulSoup(request_text, "lxml")
exhibitions = page.find_all('div', class_ = "Zz5cHg")
L =[]
for exhibition in exhibitions:
    L.append(exhibition.img.get('src'))
print(L)

r = request(L[0], stream = True)
if r.status_code == 200:                     #200 status code = OK
   with open("images/book1.jpg", 'wb') as f: 
      r.raw.decode_content = True
      shutil.copyfileobj(r.raw, f)

img = mpimg.imread('images/book1.jpg')
imgplot = plt.imshow(img)
plt.show()

['https://static.wixstatic.com/media/3f3a7f_d6b32b0656354f06b9132f271c24431c~mv2.jpeg/v1/fill/w_147,h_77,al_c,q_80,usm_0.66_1.00_0.01,blur_2,enc_auto/3f3a7f_d6b32b0656354f06b9132f271c24431c~mv2.jpeg', 'https://static.wixstatic.com/media/3f3a7f_044cb6ea035f403dbbcdf47b07f8a81a~mv2.png/v1/fill/w_49,h_49,al_c,q_85,usm_0.66_1.00_0.01,blur_2,enc_auto/3f3a7f_044cb6ea035f403dbbcdf47b07f8a81a~mv2.png', 'https://static.wixstatic.com/media/3f3a7f_b423d6bd848d42c7be90559302b65e5b~mv2.png/v1/fill/w_49,h_26,al_c,q_85,usm_0.66_1.00_0.01,blur_2,enc_auto/3f3a7f_b423d6bd848d42c7be90559302b65e5b~mv2.png', 'https://static.wixstatic.com/media/3f3a7f_84d7823d333c4fcc9194fb4445dbfae0~mv2.jpeg/v1/fill/w_147,h_77,al_c,q_80,usm_0.66_1.00_0.01,blur_2,enc_auto/3f3a7f_84d7823d333c4fcc9194fb4445dbfae0~mv2.jpeg', 'https://static.wixstatic.com/media/3f3a7f_4f631e99e059424e97566abf6db2b217~mv2.png/v1/fill/w_49,h_26,al_c,q_85,usm_0.66_1.00_0.01,blur_2,enc_auto/3f3a7f_4f631e99e059424e97566abf6db2b217~mv2.png', 'https:/

TypeError: 'module' object is not callable